# Development

In [8]:
url = 'https://www.conestogac.on.ca/fulltime/3d-computer-animation'
driver = get_driver(url)

'Creative Industries'

In [6]:
df = pd.read_csv("dataset/all_program_names.csv")

In [7]:
count = 3

In [8]:
availabilities = {}

In [25]:
def loop(availabilities, count):
    try:
        url_list = df.ProgramLink
        for url in url_list[count-3:]:
            driver.get(url)
            time.sleep(2)

            print(count, ")", url)
            availability_temp = get_availability(driver)
            if(availability_temp):
                availabilities[url] = availability_temp
                count += 1
            else:
                return [availabilities, count]
    except:
        print("----")
        return [availabilities, count]

In [ ]:
for i in range(246):
    availabilities, count = loop(availabilities, count)

3 ) https://www.conestogac.on.ca/fulltime/3d-computer-animation
4 ) https://www.conestogac.on.ca/fulltime/academic-upgrading
5 ) https://www.conestogac.on.ca/fulltime/administrative-business-management
6 ) https://www.conestogac.on.ca/fulltime/advanced-police-studies
7 ) https://www.conestogac.on.ca/fulltime/advertising-and-marketing-communications
8 ) https://www.conestogac.on.ca/fulltime/agricultural-equipment-operator
9 ) https://www.conestogac.on.ca/fulltime/animation
10 ) https://www.conestogac.on.ca/fulltime/applied-electrical-motion-and-control-management
11 ) https://www.conestogac.on.ca/fulltime/applied-energy-management
12 ) https://www.conestogac.on.ca/fulltime/applied-energy-management-renewable-energy-stream
13 ) https://www.conestogac.on.ca/fulltime/applied-manufacturing-management-automation-stream-formerly-applied-manufacturing-management-mechatronics-stream
14 ) https://www.conestogac.on.ca/fulltime/applied-manufacturing-management-design-integration-3d-printing-stream

In [ ]:
df_avaibility = pd.DataFrame(columns=["URL", "DOMESTIC", "INTERNATIONAL"])

In [ ]:
for url, availability in availabilities.items():
    row = {"URL": url}
    domestic = []
    for intake, campus, status in zip(availability["domestic"]["intake"], availability["domestic"]["campus"], availability["domestic"]["status"]):
        domestic.append(", ".join([intake, campus, status]))
    row["DOMESTIC"] = " | ".join(domestic)

    international = []
    for intake, campus, status in zip(availability["international"]["intake"], availability["international"]["campus"], availability["international"]["status"]):
        international.append(", ".join([intake, campus, status]))
    row["INTERNATIONAL"] = " | ".join(international)

    df_avaibility = df_avaibility.append(row, ignore_index=True)

In [ ]:
df_avaibility

In [ ]:
df_avaibility.to_csv("dataset/fasnshwe_careers.csv", index=False)

In [ ]:
!shutdown -s

In [19]:
for progress in range(10):
    sys.stdout.write("Download progress: %d%%   \r" % (progress))
    time.sleep(0.8)
sys.stdout.flush()

In [24]:
url = 'https://www.conestogac.on.ca/fulltime/autism-and-behavioural-science'
driver.get(url)

def get_availability(driver):
    availability = {"domestic": {"intake": [], "campus": [], "status": []}, "international": {"intake": [], "campus": [], "status": []}}

    button = find_element_n(driver, xpath_="//button[@class='btn bg-primary-dropdown text-uppercase']")
    if(button):
        button.click()
        time.sleep(1.5)
    
    div = find_element_n(driver, xpath_="//div[@class='px-0 dropdown-menu bg-primary-dropdown dropdown-menu-right mt-0 show']")
    children = None
    if(div):
        
        children = find_elements_n(div, xpath_='./*')
    
    a_list = None
    if(children):
        
        a_list = children[:-1]
    
    test = []
    if(a_list):
        for a in a_list:
            div = find_element_n(a, xpath_="//div[@class='row mx-auto']")
            if(div):
                div_list = find_elements_n(div, class_='row')
                if(div_list):
                    intake_campus = div_list[0].text.strip()
                    intake = intake_campus.split("|")[0].strip()
                    campus = intake_campus.split("|")[1].strip()
                    status = div_list[1].text.strip()
                    domestic_status = status.split("\n")[0].strip()
                    international_status = None
                    if("\n" in status):
                        international_status = status.split("\n")[1].strip()
                    
                    availability["domestic"]["intake"].append(intake)
                    availability["domestic"]["campus"].append(campus)
                    availability["domestic"]["status"].append(domestic_status)
                    
                    if(international_status):
                        availability["international"]["intake"].append(intake)
                        availability["international"]["campus"].append(campus)
                        availability["international"]["status"].append(international_status)

    return availability

get_availability(driver)

{'domestic': {'intake': ['SEP 2022', 'SEP 2022'],
  'campus': ['ONLINE', 'ONLINE'],
  'status': ['Open', 'Open']},
 'international': {'intake': [], 'campus': [], 'status': []}}

# import

In [1]:
import sqlite3
import sys
from selenium import webdriver
import time
from os import path
from tqdm import tqdm
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from tqdm import tqdm
import pandas as pd
import numpy as np
import string
import random
from bs4 import BeautifulSoup

# Browser setup

In [2]:
def chrome():
    service = Service("../drivers/chromedriver")
    driver = webdriver.Chrome(service = service)
    return driver
def opera():
    # need to change, srd
    DRIVER_PATH = "../drivers/operadriver"
    driver = webdriver.Opera(executable_path=DRIVER_PATH)
    return driver
def firefox():
    # need to change, srd
    DRIVER_PATH = "../drivers/geckodriver"
    driver = webdriver.Firefox(executable_path=DRIVER_PATH)
    return driver

In [3]:
def get_driver(url):
    driver = chrome()
    # driver = opera()
    # driver = firefox()

    driver.get(url)
    return driver

# Element Search

In [4]:
def find_element_n(root, xpath_=None, tag_=None, id_=None, class_=None, exception=False):
    tag = None
    try:
        if(xpath_):
            tag = root.find_element(by=By.XPATH, value=xpath_)
        elif(tag_):
            tag = root.find_element(by=By.TAG_NAME, value=tag_)
        elif(id_):
            tag = root.find_element(by=By.ID, value=id_)
        elif(class_):
            tag = root.find_element(by=By.CLASS_NAME, value=class_)                
        return tag
    except:
        pass
    if(exception):
        raise Exception("\n\n\n>>> Element not found <<<\n\n\n")
    else:
        return False    

def find_elements_n(root, xpath_=None, tag_=None, id_=None, class_=None, exception=False):
    tags = None
    try:
        if(xpath_):
            tags = root.find_elements(by=By.XPATH, value=xpath_)
        elif(tag_):
            tags = root.find_elements(by=By.TAG_NAME, value=tag_)
        elif(id_):
            tags = root.find_elements(by=By.ID, value=id_)
        elif(class_):
            tags = root.find_elements(by=By.CLASS_NAME, value=class_)                
        return tags
    except:
        pass
    if(exception):
        raise Exception("\n\n\n>>> Element not found <<<\n\n\n")
    else:
        return False

def find_element(root, xpath_=None, tag_=None, id_=None, class_=None, init_wait = 0, wait = 1, limit=3.5, exception=False):
    """
    {init_wait} : sleep for given time before element existence check
    {wait} : sleep every time when through except except block
    {exception} : whether to through error{True} or return False{False}
    """
    time.sleep(init_wait)
    while(limit > 0):
        tag = None
        try:
            if(xpath_):
                tag = root.find_element(by=By.XPATH, value=xpath_)
            elif(tag_):
                tag = root.find_element(by=By.TAG_NAME, value=tag_)
            elif(id_):
                tag = root.find_element(by=By.ID, value=id_)
            elif(class_):
                tag = root.find_element(by=By.CLASS_NAME, value=class_)                
            return tag
        except:
            print("----", limit, "----")
            time.sleep(wait)
            limit -= wait
    if(exception):
        raise Exception("\n\n\n>>> Element not found <<<\n\n\n")
    else:
        return False

def find_elements(root, xpath_=None, tag_=None, id_=None, class_=None, init_wait = 0, wait = 1, limit=3.5, exception=False):
    """
    {init_wait} : sleep for given time before element existence check
    {wait} : sleep every time when through except except block
    {exception} : whether to through error{True} or return False{False}
    """
    time.sleep(init_wait)
    while(limit > 0):
        tags = None
        try:
            if(xpath_):
                tags = root.find_elements(by=By.XPATH, value=xpath_)
            elif(tag_):
                tags = root.find_elements(by=By.TAG_NAME, value=tag_)
            elif(id_):
                tags = root.find_elements(by=By.ID, value=id_)
            elif(class_):
                tags = root.find_elements(by=By.CLASS_NAME, value=class_)                
            return tags
        except:
            print("----", limit, "----")
            time.sleep(wait)
            limit -= wait
    if(exception):
        raise Exception("\n\n\n>>> Element not found <<<\n\n\n")
    else:
        return False

# All Program Search

In [5]:
url = "https://www.conestogac.on.ca/fulltime"

driver = get_driver(url)

In [10]:
def get_all_program_names_list(driver):
    ul = find_element_n(driver, xpath_="//ul[@class='list-unstyled mt-md-5']")
    li_list = find_elements_n(ul, tag_='li')
    
    program_names = []
    program_links = []
    
    for li in tqdm(li_list):
        a = find_element(li, tag_='a')
        link = a.get_attribute("href").strip()        

        span = find_element(a, tag_='span')
        name = span.text.strip()
        
        program_names.append(name)
        program_links.append(link)
    
    if(program_names or program_links or program_codes):
        programs = {'ProgramName': program_names, 'ProgramLink': program_links}
        return programs
    else:
        raise Exception("---ERROR IN FETCHING PROGRAM NAME---")
        return False

programs = get_all_program_names_list(driver)

100%|██████████| 245/245 [00:14<00:00, 16.62it/s]


In [14]:
df = pd.DataFrame(programs)

In [15]:
df.to_csv('dataset/all_program_names.csv')

# Functions

## Program Name

In [23]:
def get_program_name(driver):
    div = find_element_n(driver, xpath_="//div[@class='col-12 text-white text-shadow']")
    h1 = find_element_n(div, tag_='h1')
    
    name = ""
    if(h1):
        name = h1.text.strip()
    return name
get_program_name(driver)

'3D Computer Animation'

## overview

In [26]:
def get_overview(driver):
    div = find_element_n(driver, xpath_="//div[@class='pr-lg-3']")
    children = find_elements_n(div, xpath_='./*')
    
    p = None
    for tag in children:
        if(tag.tag_name == "p"):
            p = tag
            break
    
    overview = ""
    if(p):
        overview = p.text.strip()
    return overview

get_overview(driver)

'Computer animation has evolved to become a major component in the field of animation production and contributes towards shaping content for film, television, games and online applications. As such, students must be actively engaged in all aspects of the production process and understand the requirements associated with the production pipeline. Traditional design, drawing and storytelling skills are enhanced via the latest computer (software/hardware), combined with 3D animation skills and the creation of characters and customized environments contribute towards the creation of unique content that graduates will utilize in their e-portfolio of project work.'

## category

In [7]:
def get_category(driver):
    div = find_element_n(driver, xpath_="//div[@class='my-5 border pl-3 pr-3']")
    children = find_elements_n(div, xpath_='./*')

    p = None
    for tag in children:
        if(tag.tag_name == "p"):
            p = tag
            break
    
    category = ""
    if(p):
        a = find_element_n(p, tag_='a')
        if(a):
            category = a.text.strip()
    return category

get_category(driver)

'Creative Industries'

## code

In [45]:
def get_code(driver):
    div_main = find_element_n(driver, xpath_="//div[@class='container mt-5 datasection bg-white']")
    div = find_element_n(div_main, tag_='div')
    sub_div = find_element_n(div, class_='col-lg-5')
    sub_div_1 = find_element_n(sub_div, tag_='div')
    sub_div_2 = find_element_n(sub_div_1, class_='row')
    div_list = find_elements_n(sub_div_2, tag_='div')
        
    code = ""
    div_credential = div_list[0]
    if(div_credential):
        strong = find_element_n(div_credential, tag_='strong')
        if(strong):
            strong_len = len(strong.text)
            code = div_credential.text[strong_len:].strip()
    return code
get_code(driver)

'1391'

## availability (srd working)

In [43]:
def get_availability(driver):
    availability = {"domestic": {"intake": [], "campus": [], "status": []}, "international": {"intake": [], "campus": [], "status": []}}

    button = find_element_n(driver, xpath_="//button[@class='btn bg-primary-dropdown text-uppercase']")
    if(button):
        button.click()
        time.sleep(1.5)
    
    div = find_element_n(driver, xpath_="//div[@class='px-0 dropdown-menu bg-primary-dropdown dropdown-menu-right mt-0 show']")
    print(div)
    children = None
    if(div):
        
        children = find_elements_n(div, xpath_='./*')
    
    a_list = None
    if(children):
        
        a_list = children[:-1]
    
    test = []
    if(a_list):
        for a in a_list:
            div = find_element_n(a, xpath_="//div[@class='row mx-auto']")
            if(div):
                div_list = find_elements_n(div, class_='row')
                if(div_list):
                    intake_campus = div_list[0].text.strip()
                    intake = intake_campus.split("|")[0].strip()
                    campus = intake_campus.split("|")[1].strip()
                    status = div_list[1].text.strip()
                    domestic_status = status.split("\n")[0].strip()
                    international_status = status.split("\n")[1].strip()
                    
                    availability["domestic"]["intake"].append(intake)
                    availability["domestic"]["campus"].append(campus)
                    availability["domestic"]["status"].append(domestic_status)

                    availability["international"]["intake"].append(intake)
                    availability["international"]["campus"].append(campus)
                    availability["international"]["status"].append(international_status)

    return availability

get_availability(driver)

<selenium.webdriver.remote.webelement.WebElement (session="58af2cd73cc3bc55f9a10472352d7473", element="6b6e3bd8-d35f-4140-816b-c68c4379a33a")>


{'domestic': {'intake': ['SEP 2022', 'SEP 2022'],
  'campus': ['KITCHENER – DOON', 'KITCHENER – DOON'],
  'status': ['Open', 'Open']},
 'international': {'intake': ['SEP 2022', 'SEP 2022'],
  'campus': ['KITCHENER – DOON', 'KITCHENER – DOON'],
  'status': ['Closed', 'Closed']}}

## credential

In [44]:
def get_credential(driver):
    div_main = find_element_n(driver, xpath_="//div[@class='container mt-5 datasection bg-white']")
    div = find_element_n(div_main, tag_='div')
    sub_div = find_element_n(div, class_='col-lg-5')
    sub_div_1 = find_element_n(sub_div, tag_='div')
    sub_div_2 = find_element_n(sub_div_1, class_='row')
    div_list = find_elements_n(sub_div_2, tag_='div')
        
    credential = ""
    div_credential = div_list[5]
    if(div_credential):
        strong = find_element_n(div_credential, tag_='strong')
        if(strong):
            strong_len = len(strong.text)
            credential = div_credential.text[strong_len:].strip()

    return credential

get_credential(driver)

'Ontario College Graduate Certificate'

## Program Coordinator

In [46]:
def get_coordinator(driver):

    div = find_element_n(driver, xpath_="//div[@class='my-5 border pl-3 pr-3']")
    children = find_elements_n(div, xpath_='./*')

    ul = None
    for tag in children:
        if(tag.tag_name == "ul"):
            ul = tag
            break
    
    li = None
    if(ul):
        li = find_element_n(ul, tag_='li')
    
    span = None
    if(li):
        span = find_element_n(li, tag_='span')
    
    coordinator = ""
    if(span):
        coordinator = span.text.strip()

    return coordinator

get_coordinator(driver)

'David Shepstone'

## delivery

In [53]:
def get_delivery(driver):
    div_main = find_element_n(driver, xpath_="//div[@class='container mt-5 datasection bg-white']")
    div = find_element_n(div_main, tag_='div')
    sub_div = find_element_n(div, class_='col-lg-5')
    sub_div_1 = find_element_n(sub_div, tag_='div')
    sub_div_2 = find_element_n(sub_div_1, class_='row')
    div_list = find_elements_n(sub_div_2, tag_='div')
        
    delivery = ""

    div_delivery = div_list[1]
    if(div_delivery):
        strong = find_element_n(div_delivery, tag_='strong')
        if(strong):
            strong_len = len(strong.text)
            delivery = div_delivery.text[strong_len:].strip()

    return delivery

get_delivery(driver)

'Full-time'

## Tution Fees (srd working)

In [14]:
def get_tution_fees(driver):
    fees = {"domestic_fees": "", "international_fees": ""}
    
    div = find_element_n(driver, class_='fees-block', exception=False)
    div_domestic = find_element_n(div, class_='canadian-cost-block', exception=False)
    div_international = find_element_n(div, class_='international-cost-block', exception=False)
    
    span_domestic = find_element_n(div_domestic, class_='fees-cost-dollar', exception=False)
    span_international = find_element_n(div_international, class_='fees-cost-dollar', exception=False)
    
    if(span_domestic):
        fees["domestic_fees"] = span_domestic.text.strip()
    if(span_international):
        fees["international_fees"] = span_international.text.strip()

    return fees
    
get_tution_fees(driver)

{'domestic_fees': '$4,856', 'international_fees': '$15,840'}

## Related Programs (srd working)

In [16]:
def get_related_programs(driver):
    related_programs = {}
    main_div = find_element_n(driver, class_='field--name-field-related-programs', exception=False)
    div = find_element_n(main_div, class_='field__items', exception=False)
    
    if(div):
        sub_divs = find_elements_n(div, class_='field__item')
        if(sub_divs):
            for sub_div in sub_divs:
                a = sub_div.find_element(By.TAG_NAME, 'a')
                if(a):
                    name = a.text.strip()
                    link = a.get_attribute("href").strip()
                    related_programs[name] = link
    return related_programs

get_related_programs(driver)

{'Public Relations - Corporate Communications': 'https://www.fanshawec.ca/programs/cor3-public-relations-corporate-communications',
 'Marketing Management': 'https://www.fanshawec.ca/programs/mkm1-marketing-management',
 'Advanced Communication for Professionals': 'https://www.fanshawec.ca/programs/adc1-advanced-communication-professionals'}

# Other Functions

##  url_creater_for_subjects

In [17]:
def url_creater_for_subjects(url):
    url_list = url.split("/")
    url_list = url_list[:-1]
    url_list.append("courses-next")
    url = "/".join(url_list)
    return url

## Link joiner

In [18]:
def link_joiner(url):
    base_url = "https://www.fanshawec.ca" + url
    return base_url

# Data Screapping

In [19]:
program_details = {
    "URL": None,
    "CollegeName": None,
    "ProgramName": None,
    "Overview": None,
    "Category": None,
    "Code" : None,
    "Availability" : None,
    "Credential" : None,
    "RelatedPrograms" : None,
    "ProgramCoordinator" : None,
    "Delivery" : None,
    "TuitionFees" : None,
}

In [9]:
def get_program_details(program_details, college_name, url):
    driver.get(url)
    time.sleep(0.3)
    
    program_details["URL"] = url.strip()
    program_details["CollegeName"] = college_name.strip()
    program_details["ProgramName"] = get_program_name(driver)
    program_details["Overview"] = get_overview(driver)
    program_details["Category"] = get_category(driver)
    program_details["Code"] = get_code(driver)
    program_details["Availability"] = get_availability(driver)
    program_details["Credential"] = get_credential(driver)
    program_details["RelatedPrograms"] = get_related_programs(driver)
    program_details["ProgramCoordinator"] = get_coordinator(driver)
    program_details["Delivery"] = get_delivery(driver)
    program_details["TuitionFees"] = get_tution_fees(driver)

    return program_details

In [21]:
df_all_programs = pd.read_csv('./dataset/all_program_names.csv')

In [22]:
count = 3

In [23]:
programs = {}

In [28]:
for url in df_all_programs.ProgramLink[count-3:]:
    print(str(count) + ') '+ url)
    programs[url] = get_program_details(program_details.copy(), "Fanshawe", url)
    count += 1

235) https://www.fanshawec.ca/programs/smm1-sport-and-event-marketing/next
236) https://www.fanshawec.ca/programs/scm2-supply-chain-management-logistics-co-op/next
237) https://www.fanshawec.ca/programs/tes2-teaching-english-speakers-other-languages-and-intercultural-competence/next
238) https://www.fanshawec.ca/programs/tss2-technical-systems-analysis/next
239) https://www.fanshawec.ca/programs/tdm2s-tool-and-die-maker-block-and-day-release-apprenticeship/next
240) https://www.fanshawec.ca/programs/ttc6-tourism-travel/next
241) https://www.fanshawec.ca/programs/tts1-tourism-travel-studies/next
242) https://www.fanshawec.ca/programs/tct4-truck-and-coach-technician-block-release-apprenticeship/next
243) https://www.fanshawec.ca/programs/tct3-truck-and-coach-technician-day-release-apprenticeship/next
244) https://www.fanshawec.ca/programs/vee1-visual-effects-and-editing-contemporary-media/next
245) https://www.fanshawec.ca/programs/iwd2-web-development-and-internet-applications/next
246)

# Store to DataFrame

In [162]:
columns = ["ID", "URL", "College", "Program", "Overview", "Category", "Code", "Domestic Availability", "International Availability", "Credential", "Related Programs", "Program Coordinator", "Delivery", "Domestic Fees", "International Fees"]

In [163]:
columns

['ID',
 'URL',
 'College',
 'Program',
 'Overview',
 'Category',
 'Code',
 'Domestic Availability',
 'International Availability',
 'Credential',
 'Related Programs',
 'Program Coordinator',
 'Delivery',
 'Domestic Fees',
 'International Fees']

In [164]:
L = len(programs.keys())
program_id_list = {}

letters = string.ascii_uppercase + string.digits

for key in programs.keys():
    while(True):
        temp_id = "FAN"+''.join(random.choice(letters) for i in range(4))
        if(temp_id not in program_id_list):
            program_id_list[temp_id] = key
            break

In [165]:
def get_value_from_string(string):
    if(string):
        return string
    return np.NaN

def make_string_availability(availability):
    intake = availability["intake"]
    campus = availability["campus"]
    status = availability["status"]
    temp = [intake_ + ", " + campus_ + ", " + status_ for intake_, campus_, status_ in zip(intake, campus, status)]
    availability = " | ".join(temp)
    return availability

In [166]:
df = pd.DataFrame(columns=columns)

for id_, url in program_id_list.items():
    row = {}
    row = {"ID": id_, "URL": url}
    
    program = programs[url]
    
    row["College"] = program["CollegeName"]
    row["Program"] = get_value_from_string(program["ProgramName"])
    row["Overview"] = get_value_from_string(program["Overview"])
    row["Category"] = get_value_from_string(program["Category"])
    row["Code"] = get_value_from_string(program["Code"])
    
    availability_domestic_temp = program["Availability"]["domestic"]
    availability_domestic_temp = make_string_availability(availability_domestic_temp)
    row["Domestic Availability"] = get_value_from_string(availability_domestic_temp)

    availability_international_temp = program["Availability"]["international"]
    availability_international_temp = make_string_availability(availability_international_temp)
    row["International Availability"] = get_value_from_string(availability_international_temp)
    
    row["Credential"] = get_value_from_string(program["Credential"])
    
    related_temp = program["RelatedPrograms"].keys()
    related_temp = ", ".join(related_temp)
    row["Related Programs"] = get_value_from_string(related_temp)
    
    row["Program Coordinator"] = get_value_from_string(program["ProgramCoordinator"])
    
    delivery_temp = program["Delivery"]
    delivery_temp = ", ".join(delivery_temp.split("\n"))
    row["Delivery"] = get_value_from_string(delivery_temp)
    
    row["Domestic Fees"] = get_value_from_string(program["TuitionFees"]["domestic_fees"])
    row["International Fees"] = get_value_from_string(program["TuitionFees"]["international_fees"])
    
    df = df.append(row, ignore_index=True)

In [168]:
df

,ID,URL,College,Program,Overview,Category,Code,Domestic Availability,International Availability,Credential,Related Programs,Program Coordinator,Delivery,Domestic Fees,International Fees
0,FANRW4M,https://www.fanshawec.ca/programs/dcm2-digital...,Fanshawe,Digital Communications Management,Pursue a career in communications and add prac...,School of Contemporary Media,DCM2,"2022 September, London, Open | 2023 January, L...","2023 January, London, Open",Ontario College Graduate Certificate,"Public Relations - Corporate Communications, M...",Jackie Westelaken,Full-Time,"$4,856","$15,840"
1,FANZAFL,https://www.fanshawec.ca/programs/esd3-electro...,Fanshawe,Electronics and Embedded Systems Development,Electronics and Embedded Systems Development i...,School of Applied Science and Technology,ESD3,"2022 September, London, Open | 2023 January, L...","2022 September, London, Open | 2023 January, L...",Ontario College Graduate Certificate,Electromechanical Engineering Technician (Co-op),Chris Talbot,"In Class, Full-Time",NaN,"$34,029"
2,FANG1SK,https://www.fanshawec.ca/programs/anc1-3d-anim...,Fanshawe,3D Animation and Character Design,"Characters are at the heart of every film, vid...",School of Digital and Performing Arts,ANC1,"2022 September, London - Downtown, Open",NaN,Ontario College Graduate Certificate,"Photography, Graphic Design, Interactive Media...",David Latour,"Day Time, Full-Time","$7,307","$16,040"
3,FANIDUP,https://www.fanshawec.ca/programs/psc1-911-and...,Fanshawe,911 and Public Safety Communications,"Emergency communications professionals, includ...",School of Public Safety,PSC1,"2022 September, London, Open","2022 September, London, Open",Ontario College Certificate,NaN,Tracy Dietrich,"Day Time, Full-Time","$4,287","$15,641"
4,FANS2I7,https://www.fanshawec.ca/programs/ats1-acting-...,Fanshawe,Acting for Screen and Stage,"Do you have a passion for drama, dance, music ...",School of Contemporary Media,ATS1,"2022 September, London, Waitlisted",NaN,Ontario College Diploma,"Advanced Filmmaking, Broadcasting - Television...",NaN,"Day Time, Full-Time","$8,533","$33,856"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,FANELFU,https://www.fanshawec.ca/programs/tct3-truck-a...,Fanshawe,Truck and Coach Technician (Day Release) [Appr...,The Truck and Coach Technician Apprenticeship ...,School of Transportation Technology and Appren...,TCT3,NaN,NaN,Ontario College Certificate,NaN,Mark Russell,Apprenticeship,NaN,NaN
241,FANC2WO,https://www.fanshawec.ca/programs/vee1-visual-...,Fanshawe,Visual Effects and Editing for Contemporary Media,"If you have a background in graphic design, ph...",School of Contemporary Media,VEE1,"2022 September, London, Open","2022 September, London, Open",Ontario College Graduate Certificate,"3D Animation and Character Design, Interactive...",Scott Doerr,"Day Time, Full-Time","$5,677","$16,728"
242,FANHID7,https://www.fanshawec.ca/programs/iwd2-web-dev...,Fanshawe,Web Development and Internet Applications,Today's web development professional requires ...,School of Information Technology,IWD2,"2022 September, London - Downtown, Open | 2023...","2023 January, London - Downtown, Open",Ontario College Diploma,Computer Programming and Analysis,Stuart Budden,"Day Time, Full-Time, Part-Time, Online","$7,206","$30,699"
243,FANSHLE,https://www.fanshawec.ca/programs/wft1-welding...,Fanshawe,Welding and Fabrication Technician (Co-op),Welding and Fabrication Technician is a two-ye...,School of Applied Science and Technology,WFT1,"2022 September, London, Waitlisted | 2023 Janu...","2022 September, London, Open | 2023 January, L...",Ontario College Diploma,Welding Techniques,Program Coordinator,"Co-op, Day Time, Full-Time","$6,831","$31,324"


# Create CSV file

In [169]:
df.to_csv("dataset//fanshawe_dataset.csv", index=False)

# Careers

In [124]:
def get_careers(driver):
    a = driver.find_element(by=By.CLASS_NAME, value='details-title')

    if(a):
        a.click()
        time.sleep(2)

    source = driver.page_source
    soup = BeautifulSoup(source, 'lxml')

    div = soup.find('details', class_='collapse-processed')
    p_list = div.find_all("p")

    careers = []
    for p in p_list:
        career = p.find("strong")
        if(career):
            careers.append(career.text.strip())
    return careers
get_careers(driver)

['Digital Communication Specialist',
 'Social Media Manager',
 'Digital Media Manager']

In [71]:
count = 3

In [72]:
career_list = {}

In [10]:
key_values = list(program_id_list.items())
for id_, url in key_values[count-3:]:
    driver.get(url)
    time.sleep(0.3)
    
    print(count, ")", url)
    careers = get_careers(driver)
    career_list[id_] = ", ".join(careers)
    count += 1

NameError: name 'program_id_list' is not defined

In [131]:
def loop(career_list, count):
    try:
        key_values = list(program_id_list.items())
        for id_, url in key_values[count-3:]:
            driver.get(url)
            time.sleep(2)

            print(count, ")", url)
            careers = get_careers(driver)
            career_list[id_] = ", ".join(careers)
            count += 1
    except:
        print("----")
        return [career_list, count]

In [134]:
for i in range(260):
    career_list, count = loop(career_list, count)

208 ) https://www.fanshawec.ca/programs/plq1-plumbing-techniques/next
209 ) https://www.fanshawec.ca/programs/pft1-police-foundations/next
----
209 ) https://www.fanshawec.ca/programs/pft1-police-foundations/next
210 ) https://www.fanshawec.ca/programs/pft2w-police-foundations-accelerated/next
----
210 ) https://www.fanshawec.ca/programs/pft2w-police-foundations-accelerated/next
211 ) https://www.fanshawec.ca/programs/peq2s-power-engineering-techniques-4th-class/next
----
211 ) https://www.fanshawec.ca/programs/peq2s-power-engineering-techniques-4th-class/next
212 ) https://www.fanshawec.ca/programs/pem1-practical-elements-mechanical-engineering/next
----
212 ) https://www.fanshawec.ca/programs/pem1-practical-elements-mechanical-engineering/next
213 ) https://www.fanshawec.ca/programs/png5-practical-nursing/next
----
213 ) https://www.fanshawec.ca/programs/png5-practical-nursing/next
214 ) https://www.fanshawec.ca/programs/phs2-pre-health-sciences-pathway-advanced-diplomas-and-degrees/

TypeError: cannot unpack non-iterable NoneType object

In [158]:
df_careers = pd.DataFrame(columns=["ID", "Careers"])

In [159]:
for id_, careers in career_list.items():
    row = {"ID": id_, "Careers": careers.strip(' ,')}
    df_careers = df_careers.append(row, ignore_index=True)

In [161]:
df_careers.to_csv("dataset/fasnshwe_careers.csv", index=False)